In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv")
df2=pd.read_csv("/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv")

In [3]:
df.sample()

,clean_comment,category
30311,funny might also criminal also unacceptable de...,-1


In [4]:
df["clean_text"]=df["clean_comment"]
df["target"]=df["category"]

In [5]:
df=df.drop(columns=["clean_comment","category"],axis=1)

In [6]:
df

,clean_text,target
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


In [7]:
df.size

74498

In [8]:
df2["clean_text"][12]

'calm waters wheres the modi wave '

In [9]:
df2["target"]=df2["category"]

In [10]:
df2=df2.drop(columns=["category"],axis=1)

In [11]:
df2

,clean_text,target
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [12]:
df2.shape

(162980, 2)

In [13]:
df2.size

325960

In [14]:
new_df=pd.concat([df,df2],ignore_index=True)

In [15]:
new_df.sample()

,clean_text,target
91377,after india becomes space superpower modi list...,1.0


In [16]:
new_df.size

400458

In [17]:
new_df.shape

(200229, 2)

In [18]:
new_df["clean_text"][100]

' want move gujarat and live there for few months any city that you can recommend '

In [19]:
new_df.isnull().sum()

clean_text    104
target          7
dtype: int64

In [20]:
from sklearn.impute import SimpleImputer

In [21]:
sim=SimpleImputer(strategy="constant",fill_value=np.nan)
sim.fit(new_df)
data=sim.transform(new_df)
data=pd.DataFrame(data)
data.columns

RangeIndex(start=0, stop=2, step=1)

In [22]:
data

,0,1
0,family mormon have never tried explain them t...,1.0
1,buddhism has very much lot compatible with chr...,1.0
2,seriously don say thing first all they won get...,-1.0
3,what you have learned yours and only yours wha...,0.0
4,for your own benefit you may want read living ...,1.0
...,...,...
200224,why these 456 crores paid neerav modi not reco...,-1.0
200225,dear rss terrorist payal gawar what about modi...,-1.0
200226,did you cover her interaction forum where she ...,0.0
200227,there big project came into india modi dream p...,0.0


In [23]:
data.isnull().sum()

0    104
1      7
dtype: int64

# # Model

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
new_df

,clean_text,target
0,family mormon have never tried explain them t...,1.0
1,buddhism has very much lot compatible with chr...,1.0
2,seriously don say thing first all they won get...,-1.0
3,what you have learned yours and only yours wha...,0.0
4,for your own benefit you may want read living ...,1.0
...,...,...
200224,why these 456 crores paid neerav modi not reco...,-1.0
200225,dear rss terrorist payal gawar what about modi...,-1.0
200226,did you cover her interaction forum where she ...,0.0
200227,there big project came into india modi dream p...,0.0


In [26]:
X=data[0]
Y=data[1]

In [27]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [28]:
Y_test

194517    1.0
147887    1.0
87523     1.0
45173     0.0
99538     0.0
         ... 
115738    1.0
100763    1.0
143288    0.0
178467    1.0
54271    -1.0
Name: 1, Length: 60069, dtype: object

In [29]:
len(X_test)
X_test.isnull().sum()

32

In [30]:
X_train2=X_train.dropna()
X_test2=X_test.dropna()
Y_test2=Y_test.dropna()
Y_train2=Y_train.dropna()
X_train2.isnull().sum()
len(X_train2)
len(X_test2)
Y_test2.isnull().sum()

0

In [31]:
Y_train

75774    -1.0
3958      1.0
130767    0.0
182973    1.0
198659    1.0
         ... 
119879    1.0
103694    0.0
131932    0.0
146867    1.0
121958    1.0
Name: 1, Length: 140160, dtype: object

In [32]:
vectorize=TfidfVectorizer()
X_train_vt=vectorize.fit_transform(X_train2)



In [33]:
X_test_vt=vectorize.transform(X_test2)

In [34]:
import pandas as pd

# Assuming X_train_vt is your sparse matrix
X_train_vt_df = pd.DataFrame.sparse.from_spmatrix(X_train_vt, columns=vectorize.get_feature_names_out())


In [35]:
X_test_vt_df = pd.DataFrame.sparse.from_spmatrix(X_test_vt, columns=vectorize.get_feature_names_out())

In [36]:
X_train_vt_df.shape

(140088, 106053)

In [37]:
# X_train_vt1=pd.DataFrame(X_train_vt)

In [38]:
# X_train_vt1.shape

In [39]:
X_train_vt.shape

(140088, 106053)

In [40]:
X_train_vt.shape

(140088, 106053)

In [41]:
Y_train

75774    -1.0
3958      1.0
130767    0.0
182973    1.0
198659    1.0
         ... 
119879    1.0
103694    0.0
131932    0.0
146867    1.0
121958    1.0
Name: 1, Length: 140160, dtype: object

In [42]:
Y_train2=Y_train2.iloc[0:140088]
# y=Y_train.iloc[:,:]
Y_train2=Y_train2.astype(float)
# Y_train2.str.astype(int)
Y_train2.shape

(140088,)

In [43]:
X_train_vt_df.shape

(140088, 106053)

In [44]:
X_train_vt

<140088x106053 sparse matrix of type '<class 'numpy.float64'>'
	with 2675789 stored elements in Compressed Sparse Row format>

In [45]:
um_samples = 20
sha=(um_samples,140088,106053)

In [46]:
# X_train_vt.reshape(sha)

In [47]:
# X_train_vt.shape

In [48]:
# X_test_vt.shape

In [49]:
Y_test2=Y_test2.iloc[0:60037]
Y_test2=Y_test2.astype(float)
# Y_test2

In [50]:
# score=random_for.score(X_test_vt_df,Y_test2)
# prediction=random_for.predict(X_test_vt_df)


In [51]:
# score

In [52]:
# accuracy=accuracy_score(X_test,Y_predict)

In [53]:
# prediction

In [54]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,SimpleRNN,GRU,Dropout,Flatten,Input
from keras.preprocessing.sequence import pad_sequences

2024-05-25 18:27:54.225520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 18:27:54.225566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 18:27:54.227118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [55]:
import tensorflow as tf

# Define the EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   # Quantity to be monitored (e.g., validation loss)
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    verbose=1,            # Verbosity mode (0: silent, 1: update messages)
    restore_best_weights=True  # Restore the model weights from the epoch with the best value of the monitored quantity
)


In [56]:
X_test1=X_test_vt.toarray()
X_train1=X_train_vt.toarray()
Y_train1=Y_train2
Y_test1=Y_test2

In [57]:
X_test3=X_test_vt[0:5000].toarray()
X_train3=X_train_vt[0:5000].toarray()
Y_train3=Y_train2[0:5000]
Y_test3=Y_test2[0:5000]

In [58]:
X_test4=X_test_vt[5000:15000].toarray()
X_train4=X_train_vt[5000:15000].toarray()
Y_train4=Y_train2[5000:15000]
Y_test4=Y_test2[5000:15000]

In [59]:
# random_for=RandomForestClassifier()
# random_for.fit(X_train3,Y_train3)

In [60]:
# random_for.score(X_test3,Y_test3)

In [61]:
from sklearn.linear_model import LinearRegression,RidgeClassifier,RidgeClassifierCV,LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [62]:
# base_estimator=[
#     ("rid_class",RidgeClassifier()),
#     ("ridgeClasf_cv",RidgeClassifierCV()),
#     ("mlp_clasf",MLPClassifier()),
#     ("gaus_nb",GaussianNB()),
#     ("k_neigh_clasf",KNeighborsClassifier()),
#     ("radius_clasf",RadiusNeighborsClassifier()),
#     ("catboost",CatBoostClassifier()),
#     ("xbgboost",XGBClassifier()),
#     ("decision_tree",DecisionTreeClassifier()),
#     ("random_forest",RandomForestClassifier())
    
    
# ]
# staking_algo=StackingClassifier(estimators=base_estimator,final_estimator=SVC())
# staking_algo.fit(X_train1,Y_train1)
# staking_algo.score(X_train1,Y_train1)
# staking_algo.score(X_test1,Y_test1)

In [63]:
# random_for1=RandomForestRegressor()
# random_for1.fit(X_train3,Y_train3)

NameError: name 'RandomForestRegressor' is not defined

In [ ]:
# random_for1.score(X_test3,Y_test3)

In [ ]:
# X_train_vt.shape

In [65]:
model3 =Sequential([
    Input(shape=(140088, 106053)),
    SimpleRNN(100,return_sequences=True,activation="relu"),
    Dense(100,activation="relu"),
    Dense(200,activation="relu"),
    Dense(200,activation="relu"),
    Dropout(0.5),
    SimpleRNN(400,return_sequences=True,activation="relu"),
    Dense(400,activation="relu"),
    Dense(800,activation="relu"),
    Dense(800,activation="relu"),
    Dropout(0.5),
    SimpleRNN(1600,return_sequences=True,activation="relu"),
    Dense(400,activation="relu"),
    Dense(400,activation="relu"),
    Dense(500,activation="relu"),
    Dropout(0.5),
    SimpleRNN(32,return_sequences=False,activation="relu"),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])
model3.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])


In [66]:
model3.input_shape

(None, 140088, 106053)

# # Deep learnin reshape code

In [67]:
import numpy as np

# Reshape X_train3 to match the expected input shape of your model
num_samples = X_train1.shape[0]
sequence_length = -1
input_dim = X_train1.shape[1]

X_train_reshaped = X_train1.reshape(num_samples, sequence_length, input_dim)

# Now, X_train_reshaped has the shape (num_samples, sequence_length, input_dim),
# which matches the expected input shape of your model
num_samples = X_test1.shape[0]
sequence_length = -1
input_dim = X_test1.shape[1]

X_test_reshaped = X_test1.reshape(num_samples, sequence_length, input_dim)
X_test_reshaped.shape

(60037, 1, 106053)

In [ ]:
# X_train3.shape[1]

In [ ]:
model3.fit(X_train1,Y_train1,epochs=100,batch_size=32,verbose=1,validation_data=(X_test1,Y_test1),callbacks=[early_stopping])


In [ ]:
loss3,accuracy3=model3.evaluate(X_test1,Y_test1)
print("Loss of Sequential model:",loss3)
print("accuracy of Sequential model:",accuracy3)

# # # Quanutum Neural network

In [ ]:
# # import cirq
# from sklearn.transformer import FuntionalTransformer